# Exo préliminaire

In [ ]:
beer_id2desct = {
    1: "portez ce vieux whiskey au juge blond qui fume",
    2: "On fume ce malt pour obtenir ce whiskey tourbé",
    3: "La fermentation haute donne des bières plus fruitées.", 
    4: "Ce houblon ajoute une amertume distincte à la bière.", 
    5: "Le malt apporte la rondeur à la bière mais un malt torréfié apporte des arômes de café à la bière.", 
    6: "La bière de fermentation basse est plus légère et rafraîchissante.", 
    7: "La température de fermentation influence le goût final de la bière.", 
}

# get vocabulary
vocab = sorted(set([word for doc in beer_id2desct.values() for word in doc.lower().split()]))
# compute posting list in a naïve whay
posting_list = {
    word: {
        doc_id: [i for i, w in enumerate(str(doc_str).lower().split()) if w == word]
        for doc_id, doc_str in beer_id2desct.items()
        if " " + word + " " in str(doc_str).lower()
    }
    for word in vocab
}

def fetch_doc_ids_having_word(word):
    return posting_list[word.lower()]

# Uses cases

## UC-1 : description data

Attention, plus compliqué qu'en SQL. Voir la [doc spécifique Vespa sur le grouping](https://docs.vespa.ai/en/grouping.html). Ne pas chercher à aller jusqu'au bout.

- Q1: Combien y a-t-il de bières dans la DB ?
- Q2: Top10 brasseries les plus représentées avec le nombre de bière par brasserie ?
- Q3: Top10 des bières les plus fortes (ABV) en France ?
- Q4: Par pays, nombre de brasseries qui proposent des bières de type `Porter` et ABV moyen de celles-ci ?
- Q5: Mediane du nombre de bière par pays ?

In [ ]:
import json
from vespa.application import Vespa
import json

vespa = Vespa(url="http://vespa", port=8080)
vespa.wait_for_application_up(5)


In [ ]:
# Q1
resp = vespa.query(
    {
        "yql": "select * from beer where true limit 0 | all( output(count()) )", 
        "hits": 10
    }
)
print(json.dumps(resp.json["root"]["children"], indent=2))

In [ ]:
# Q2
resp = vespa.query(
    {
        "yql": "select * from beer where true limit 0 | all( group(brewery) order(-count()) each(output(count())) )", 
        "hits": 10
    }
)
print(json.dumps(resp.json["root"]["children"][0]["children"][0]["children"], indent=2))

In [ ]:
# Q3
resp = vespa.query(
    {
        "yql": "select * from beer where country contains 'France' | all( group(name) order(-max(abv)) each(output(max(abv))) )", 
        "hits": 10
    }
)
print(json.dumps(resp.json["root"]["children"][0]["children"][0]["children"], indent=2))

In [ ]:
# Q4 : double groupby horrible
# Q5 : pas de médiane

Observation: c'est l'enfer avec Vespa !

# UC-2 : préparer un dataset de ranking 

Plusieurs problèmes pour résoudre ce use-case:
- cf intro : un index ne peut pas faire de jointure 
- cf UC-1 : les grouping sont horribles à réaliser

Conclusion : les index ne sont vraiment pas adaptés pour fabriquer des datasets !

# UC-3 : récupérer les docs qui parlent d'un mot

Peut-on utiliser SQL pour réaliser un mini moteur de recherche ? Pour différentes requêtes (`query` en anglais) textuelles très simples à base de mot-clef, retrouver les bières qui semblent répondre à la demande. Exemples :
- trouver les bières ou les brasseries qui parlent de bières "fine"
- idem pour "juicy"
- idem pour "genuine"
- idem pour les bières mâturées dans des "oak cask" (fûts en chêne) -> combien y en a-t-il ? $N_1$
   - idem pour les bières qui évoquent uniquement "cask" -> combien y en a-t-il ? $N_{1,1}$
   - idem pour celles ne parlant que de "oak" -> combien y en a-t-il ? $N_{1,2}$
- idem pour les bières qui évoquent "oak" et "cask" -> combien y en a-t-il ? $N_{2}$

In [ ]:
query = "oak cask"
resp = vespa.query(
    {
        "hits": 2, 
        "query": query, 
        "ranking.profile": "rank-brewery-and-descr",
    }
)
print(json.dumps(resp.json["root"]["children"], indent=2))

# UC - 5 réponse sur un corpus ?

In [ ]:
query = "child friendly beer"
resp = vespa.query(
    {
        "yql": "select * from beer where {targetHits:3}nearestNeighbor(mrl_embedding, q)",
        "input.query(q)": "embed(mxbai, @text)",
        "ranking.profile": "ann",
        "text": query,
        "presentation.summary": "textual"
    }
)
print(json.dumps(resp.json["root"]["children"], indent=2))